## Setup

In [2]:
! pip install openai python-dotenv elasticsearch


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
api_key = os.getenv("OPENAI_API_KEY")

In [6]:
#os.environ

In [7]:
client = OpenAI(api_key=api_key)

In [64]:
#Example:
#response = client.chat.completions.create(
#    model = 'gpt-3.5-turbo',
#    messages = [{"role": "user", "content": "is it too late to join the course?"}]
#)
#response.choices[0].message.content

## Simple RAG

In [8]:
import json
import minsearch

In [9]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [10]:
documents = []

#docs_raw[2] # 3 courses
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [11]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

create an instance fo the index class, classifying the text and keyword fields  
(Filter by keyword rtext field)

In [12]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields=["course"]
)

In [13]:
q = 'The course has already started, can I still enroll?'

In [14]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [15]:
index.fit(documents)

In [18]:
boost = { 
    'question': 3.0,
    'section': 0.5

}
results = index.search(
    query = q,
    boost_dict= boost, 
    num_results = 5
)
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [20]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages = [{
        "role": "user",
        "content": q
    }]
)

In [25]:
response.choices[0].message.content

'It may be possible to still enroll in a course that has already started, but it ultimately depends on the policies of the educational institution or program offering the course. You should reach out to the institution or program directly to inquire about their enrollment deadlines or policies for late enrollment. They may have provisions for late enrollment or the possibility of joining the course at a later date.'

In [34]:
prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ databse. 
Use only the facts in the CONTEXT when answering the question.
If the context doesn't contain the answer, output NONE.

QUESTION: 
{question}

CONTEXT: 
{context}
""".strip()

In [35]:
context = ""

for doc in results:
    context += f"section: {doc['section']} \nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [31]:
print(context)

section: General course-related questions 
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions 
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions 
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so yo

In [38]:
prompt = prompt_template.format(question=q, context = context).strip()

In [39]:
response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{"role": "user", "content": prompt}]
)

In [40]:
response.choices[0].message.content

"Yes, you can still enroll in the course after it has already started. You won't be able to submit some of the homeworks, but you can still take part in the course. In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ projects by the deadline. If you join the course late and manage to work on two projects, you will still be eligible for a certificate."

## Modularization

In [42]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    results = index.search(
        query = query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )   
    return results



In [45]:
#search("how do I run kafka?")

In [57]:
def build_prompt(query, search_results):
    prompt_template = """
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ databse. 
    Use only the facts in the CONTEXT when answering the question.

    QUESTION: 
    {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context += f"section: {doc['section']} \nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context = context).strip()

    return prompt

In [58]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [59]:
query = "how do I run kafka?"
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

In [61]:
print(answer)

To run Kafka, you can follow these different methods based on the language you are using:
1. For Java programs, in the project directory, run:
`java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`
2. For Python programs, such as producer.py, you can create a virtual environment and run requirements.txt and the python files in that environment by following these steps:
- To create a virtual env and install packages (run only once):
```
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
```
- To activate the virtual environment (you'll need to run it every time you need the virtual env):
```
source env/bin/activate
```
- To deactivate the virtual environment:
```
deactivate
```
3. If you encounter a permission denied error like "./build.sh: Permission denied", you can resolve it by running this command in the terminal in the same directory (/docker/spark):
`chmod +x build.sh`
4. If you face the error "ModuleNotFoundEr

## Using elastic search

Command to run elastic search  
docker run -it --name elasticsearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "xpack.security.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.4.3

http get request to local host  
curl http://localhost:9200

In [62]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [65]:
from elasticsearch import Elasticsearch

In [68]:
es_client = Elasticsearch('http://localhost:9200')

In [69]:
es_client.info()

ObjectApiResponse({'name': '2f8406804a1d', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'D4r0XbV1R7-AtjcrBH32rA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [72]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

In [73]:
index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [74]:
from tqdm.auto import tqdm 

In [76]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [00:20<00:00, 47.07it/s]


In [77]:
query = 'I just discovered the course, can I still join it?'

In [96]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs


In [97]:
results = elastic_search(query)

In [102]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [103]:
response = rag(query)

In [104]:
response

'Based on the context provided, you can still join the course even after the start date. You will be eligible to submit homeworks, but make sure to be aware of the deadlines for turning in final projects. You can also follow the course at your own pace after it finishes and continue looking at the materials and preparing for the next cohort. Additionally, before the course starts, you can install and set up all the necessary dependencies and requirements listed in the prerequisites and syllabus.'